In [1]:
import pathlib

import numpy as np
import pandas as pd
import umap

In [2]:
# set paths
# input path
data_path = pathlib.Path(
    "../../1.scDINO_run/outputdir/apoptosis_timelapse/CLS_features/CLS_features_annotated.parquet"
).resolve(strict=True)
# output path
output_path = pathlib.Path(
    "../../1.scDINO_run/outputdir/apoptosis_timelapse/CLS_features/CLS_features_annotated_umap.csv"
).resolve()
# shiny output path
shiny_output_path = pathlib.Path(
    "../temporal_shiny_app/CLS_features_annotated_umap.csv"
).resolve()

output_path.parent.mkdir(parents=True, exist_ok=True)
shiny_output_path.parent.mkdir(parents=True, exist_ok=True)

In [3]:
# load in data
cls_df = pd.read_parquet(data_path)
cls_df.head()

,Metadata_image_path,Metadata_Well,Metadata_FOV,Metadata_Time,Metadata_ImageNumber,Metadata_Nuclei_Number_Object_Number,Metadata_compound,Metadata_dose,Metadata_control,channel488-1_cls_feature_0,...,channel_DNA_cls_feature_90,channel_DNA_cls_feature_91,channel_DNA_cls_feature_92,channel_DNA_cls_feature_93,channel_DNA_cls_feature_94,channel_DNA_cls_feature_95,channel_DNA_cls_feature_96,channel_DNA_cls_feature_97,channel_DNA_cls_feature_98,channel_DNA_cls_feature_99
0,../0.pre-process_images/data/processed_images/...,C-02,0001,0001,1,102.0,Staurosporine,0.0,negative,-0.012335,...,0.035068,0.000629,-0.034905,0.056939,-0.077406,0.033168,0.005295,0.015653,0.036244,-0.013741
1,../0.pre-process_images/data/processed_images/...,C-02,0001,0001,1,106.0,Staurosporine,0.0,negative,-0.012160,...,-0.005525,-0.032507,-0.043178,0.013190,-0.073641,0.029824,0.032743,-0.017366,0.053009,-0.006046
2,../0.pre-process_images/data/processed_images/...,C-02,0001,0001,1,122.0,Staurosporine,0.0,negative,0.011821,...,0.015511,0.027061,0.002058,0.039717,-0.118989,0.010754,0.006840,0.048121,0.046048,0.018789
3,../0.pre-process_images/data/processed_images/...,C-02,0001,0001,1,128.0,Staurosporine,0.0,negative,-0.008769,...,-0.009289,-0.016169,-0.035088,0.037067,-0.101932,-0.030957,0.027611,0.001648,0.090710,-0.000729
4,../0.pre-process_images/data/processed_images/...,C-02,0001,0001,1,13.0,Staurosporine,0.0,negative,-0.000989,...,0.026018,-0.003725,0.002089,0.032937,-0.053131,0.023977,-0.018216,0.041922,0.045374,-0.033786


In [4]:
# get the metadata
metadata_df = cls_df.columns[cls_df.columns.str.contains("Metadata")]
metadata_df = cls_df[metadata_df]
feature_df = cls_df.drop(metadata_df.columns, axis=1)
print(f"metadata_df shape: {metadata_df.shape}")
print(f"feature_df shape: {feature_df.shape}")

metadata_df shape: (145489, 9)
feature_df shape: (145489, 1536)


In [5]:
# define the UMAP model
umap_model = umap.UMAP(
    n_components=2, random_state=0, n_neighbors=30, min_dist=0.1, metric="euclidean"
)

# fit the UMAP model
umap_embedding = umap_model.fit_transform(feature_df)
umap_embedding_df = pd.DataFrame(umap_embedding, columns=["UMAP1", "UMAP2"])
# add the metadata back
umap_embedding_df = pd.concat([metadata_df, umap_embedding_df], axis=1)
umap_embedding_df.head()

/home/lippincm/miniforge3/envs/timelapse_env/lib/python3.11/site-packages/umap/umap_.py:1945: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


,Metadata_image_path,Metadata_Well,Metadata_FOV,Metadata_Time,Metadata_ImageNumber,Metadata_Nuclei_Number_Object_Number,Metadata_compound,Metadata_dose,Metadata_control,UMAP1,UMAP2
0,../0.pre-process_images/data/processed_images/...,C-02,0001,0001,1,102.0,Staurosporine,0.0,negative,0.024859,2.434380
1,../0.pre-process_images/data/processed_images/...,C-02,0001,0001,1,106.0,Staurosporine,0.0,negative,0.487371,1.686836
2,../0.pre-process_images/data/processed_images/...,C-02,0001,0001,1,122.0,Staurosporine,0.0,negative,0.376325,2.612293
3,../0.pre-process_images/data/processed_images/...,C-02,0001,0001,1,128.0,Staurosporine,0.0,negative,-3.485496,0.117381
4,../0.pre-process_images/data/processed_images/...,C-02,0001,0001,1,13.0,Staurosporine,0.0,negative,1.068924,1.605956


In [6]:
# save the UMAP embeddings to parquet
umap_embedding_df.to_csv(output_path)

# save the UMAP embeddings to shiny app
umap_embedding_df.to_csv(shiny_output_path)